In [3]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from pandas import ExcelWriter

import PyLasMech as plm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
LogFiles=plm.FileFinder("../Data/PetrophysicsTVD_Geomech")
CMGWellFile=plm.FileFinder("../Data/CMG",fext=".xlsx")[0]

[IO] Found 9 log files


,FileName,Location
0,15_9-F-1 B_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-1 B_TVD...
1,15_9-F-1 C_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-1 C_TVD...
2,15_9-F-11 B_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-11 B_TV...
3,15_9-F-12_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-12_TVD_...
4,15_9-F-14_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-14_TVD_...
5,15_9-F-15 C_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-15 C_TV...
6,15_9-F-15 D_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-15 D_TV...
7,15_9-F-4_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-4_TVD_G...
8,15_9-F-5_TVD_Geomech.las,..\Data\PetrophysicsTVD_Geomech\15_9-F-5_TVD_G...


[IO] Found 1 log files


,FileName,Location
0,cmg_volve_wells.xlsx,..\Data\CMG\cmg_volve_wells.xlsx


In [5]:
CMGwells = pd.read_excel(CMGWellFile, sheet_name=None)
CMGWellNames=list(CMGwells.keys())
print("Found CMG wells ",CMGWellNames)

LogWells = [plm.ReadLas(log) for log in LogFiles]
LogWellNames=[os.path.split(os.path.split(path)[1])[1][5:] for path in LogFiles]
LogWellNames=[s.replace("_TVD_Geomech.las","") for s in LogWellNames]
print("Found Log wells ",LogWellNames)

Found CMG wells  ['I-F4G', 'I-F-1B', 'I-F-4', 'I-F-5', 'P-F-1C', 'P-F-5', 'P-F-11B', 'P-F-12', 'P-F-14', 'P-F-15C', 'P-F-15D', 'PIL-N', 'PIL-NW']
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-1 B_TVD_Geomech.las.....
[Warnning] No intersection! Get union!
Done!
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-1 C_TVD_Geomech.las.....
Done!
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-11 B_TVD_Geomech.las.....
Done!
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-12_TVD_Geomech.las.....
[Warnning] Data size (25898,25899) is in-comptabile with depth/depth_step!
Done!
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-14_TVD_Geomech.las.....
Done!
[IO] Reading F:\Course\PETE 4241\Projects\Data\PetrophysicsTVD_Geomech\15_9-F-15 C_TVD_Geomech.las.....
[Warnning] Data size (4345,4346) is in-comptabile with depth/depth_step!
Done!
[IO] Reading 

In [6]:
NumWells=len(LogWellNames)


NewSpreadSheet=[]
NewWellNames=[]
for i in range(NumWells): #Loop for each well
    log=LogWells[i]
    param=log.plm_param
    name=LogWellNames[i]
    
    name_CMG=plm.utils.findSimilarStr(name,CMGWellNames)
    NumBlks=len(CMGwells[name_CMG].index)
    print(i,name,name_CMG,NumBlks)
    NewWellNames.append(name_CMG)
    
    #Set up header for output spreadsheet
    GeomechVariables=["Block UBA","Grid Centroid X (m)","Grid Centroid Y (m)","Grid centroid Z(m)"]
    LogVariables=["SV","SHMAX","SHMIN","UCS","E","V","P_PORE","PORO"]
    ComparedVars=["CMG Pore Pressure (MPa)","CMG Porosity"]
    AllVariables=GeomechVariables+LogVariables+ComparedVars
    cols={var: [] for var in AllVariables}
    
    for gi in range(NumBlks): #Loop for each grid blocks (row)
        row=CMGwells[name_CMG].loc[gi,:]
        CMG_minmax=[float(row["TVD Entry (m)"]),float(row["TVD Exit (m)"])]
        #print("Blk=%s  TVD(%s)"%(row["Block UBA"],CMG_minmax))
        
        #Basic parameters
        cols["Block UBA"].append(row["Block UBA"])
        cols["Grid Centroid X (m)"].append(row["Grid Centroid X (m)"])
        cols["Grid Centroid Y (m)"].append(row["Grid Centroid Y (m)"])
        cols["Grid centroid Z(m)"].append(row["Grid centroid Z(m)"])
        
        #Geomech parameters
        LogDataIdx=np.where(  (log["TVD"]>=min(CMG_minmax)) & (log["TVD"]<=max(CMG_minmax)) )[0]        
        for var in LogVariables: #Loop for each variable (col)
            if(var in param.CurveNames):#This log has requried info
                #Remove nan index
                NonNanDataIdx=[idx for idx in LogDataIdx if np.isnan(log[var][idx])==False]
                #print(LogDataIdx,NonNanDataIdx)

                if(len(NonNanDataIdx)<=3): #thin gridblock using interpolation
                    TVD_cmg=np.average(CMG_minmax)
                    NonNanIndex=param.getCommonNonNanIndex(log,["TVD",var]) #np.interp requires non-nan value
                    var_cmg=np.interp(TVD_cmg,log["TVD"][NonNanIndex],log[var][NonNanIndex])
                    cols[var].append(var_cmg)
                    #if(var=="SV"): print("!!!Thin gridblock",gi,row["Block UBA"],var,TVD_cmg,var_cmg)
                
                else:#thick gridblock using averaging
                    #print(log[var][LogDataIdx])
                    cols[var].append(np.average(log[var][LogDataIdx]))
                    #if(var=="SV"): print("Thick gridblock",gi,row["Block UBA"],var,cols[var][-1])
        
        #Reference parameters
        cols["CMG Pore Pressure (MPa)"].append(row["Pore Pressure (MPa)"])
        cols["CMG Porosity"].append(row["Porosity"])

    #Convert dict into panda table and append at the end of the original one
    df  = pd.DataFrame.from_dict(cols,orient='index').transpose()
    NewSpreadSheet.append(df)

0 F-1 B I-F-1B 58


c:\users\binpc\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in greater_equal
c:\users\binpc\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in less_equal


1 F-1 C P-F-1C 115
UCS [Warning] has 148 NAN values in between
2 F-11 B P-F-11B 91
UCS [Warning] has 1007 NAN values in between
3 F-12 P-F-12 56
UCS [Warning] has 3 NAN values in between
4 F-14 P-F-14 123
5 F-15 C P-F-15C 53
6 F-15 D P-F-15D 86
UCS [Warning] has 584 NAN values in between
7 F-4 I-F-4 64
8 F-5 I-F-5 65


In [7]:
#Write the list of pandas df into excel
fpath="../Data/"#Save all new files at the root of petrophysics
fname="CMGWell_Geomech.xlsx"
fname=os.path.join(fpath,fname)

with ExcelWriter(fname) as writer:
    for i in range(NumWells):
        NewSpreadSheet[i].to_excel(writer,NewWellNames[i])
    writer.save()

In [37]:
print(NewWellNames[1])
NewSpreadSheet[1]

P-F-1C


,Block UBA,Grid Centroid X (m),Grid Centroid Y (m),Grid centroid Z(m),SV,SHMAX,SHMIN,UCS,E,V,P_PORE,PORO,CMG Pore Pressure (MPa),CMG Porosity
0,"57,51,6",435004,6.47896e+06,2889.19,49.0892,43.6894,41.2349,None,None,None,28.3142,0.031885,32.4904,0.171
1,"57,51,7",435003,6.47896e+06,2890.6,49.1245,43.7208,41.2646,None,None,None,28.3279,0.050604,32.5001,0.171
2,"57,51,8",435002,6.47896e+06,2892.02,49.1594,43.7518,41.2939,None,None,None,28.3415,0.053402,32.5099,0.171
3,"57,51,9",435002,6.47896e+06,2893.43,49.1042,43.7027,41.2475,None,None,None,28.3201,0.0396236,32.5196,0.171
4,"58,51,9",435050,6.47896e+06,2880.31,48.9549,43.5699,41.1221,None,None,None,28.2625,0.0403382,32.4283,0.171
5,"58,51,10",435050,6.47896e+06,2881.4,48.8747,43.4985,41.0547,None,None,None,28.2314,0.0448314,32.4359,0.171
6,"58,52,10",435052,6.47901e+06,2877.49,48.8101,43.441,41.0005,None,None,None,28.2063,0.06056,32.4095,0.171
7,"58,52,11",435051,6.47901e+06,2878.69,48.823,43.4525,41.0113,None,None,None,28.2113,0.057188,32.4177,0.175
8,"58,52,13",435050,6.47901e+06,2881.12,48.8833,43.5061,41.062,None,None,None,28.2347,0.0434088,32.4347,0.175
9,"58,52,15",435049,6.47901e+06,2883.56,48.9465,43.5624,41.1151,None,None,None,28.2592,0.0454052,32.4545,0.175


In [8]:
plm.printLas(log)

[LAS Info]
Well Name= 15_9-F-11 B
Start/Step/End Depth=(188.500000:0.100000:4770.200000)
NULL value= -999.25
Common Depth= (3227.9, 4744.7)


,Curves,Available Depth (Non-NULL),Raw Min/Max Val,Unit,Comments
0,DEPTH,"[188.5, 4770.2]","[188.5, 4770.2]",m,0 Depth
1,ABDCQF01,"[3227.9, 4747.3]","[1.513, 3.362]",g/cm3,1 ABDCQF01:COMPOSITE:rC:NONE:v1
2,ABDCQF02,"[3227.9, 4747.3]","[1.581, 3.18]",g/cm3,2 ABDCQF02:COMPOSITE:rC:NONE:v1
3,ABDCQF03,"[3227.9, 4747.3]","[1.539, 3.402]",g/cm3,3 ABDCQF03:COMPOSITE:rC:NONE:v1
4,ABDCQF04,"[3227.9, 4747.3]","[1.594, 3.252]",g/cm3,4 ABDCQF04:COMPOSITE:rC:NONE:v1
5,BS,"[188.5, 4770.2]","[8.5, 36.0]",inches,5 BS:COMPOSITE:rC:NONE:v1
6,CALI,"[3192.5, 4746.7]","[8.318, 10.387]",inches,6 CALI:COMPOSITE:rC:NONE:v1
7,DRHO,"[3193.6, 4747.3]","[-0.04, 0.201]",g/cm3,7 DRHO:COMPOSITE:rC:NONE:v1
8,GR,"[188.5, 4759.3]","[0.515, 170.757]",API,8 GR:COMPOSITE:rC:NONE:v1
9,NPHI,"[3198.2, 4744.7]","[0.024, 0.541]",v/v,9 NPHI:COMPOSITE:rC:NONE:v1
